# Benchmark calculcations for January and June proposals

**A Note About Source Data**

We ran our original calculations using RMI's building data. The first set of models run in this notebook used the publicly available building data from 2019, with calculations to reproduce the new fields created in the RMI dataset. This gave us massively skewed results. We're not sure if this is because the datasets were really off from one another or there's a calculation error. This will need to be investigated.

At the bottom of the notebook are these same proposals remodelled with RMI's building data, which shows us results consistent with what we've seen in the past. 


**Assumptions**:

- Buildings will produce the max amount of emissions, **unless** their predicted emissions is lower than the target GHGI, in which case they will produce that amount.
- Predicted emissions are calculated based off of the emissions predictions found in [the emissions data input file](../data/input_data/energy_emissions.csv). 


For each building, in each year (2027-2050), we need to calculate (column references given refer to the 2027 calculations in [the original RMI spreadsheet](https://docs.google.com/spreadsheets/d/175uipAHHQHGelq7i1n9sKWQXNQi-B1IJiF6-XQRVnE8/edit#gid=1811888818):

- `city_ghgi_target`: The GHGI target, as calculated by the sum of: use type * city's target GHGI for that use type * percent of GFA for that use type, for each of the three given use types (col A). If there is no target for any of the use types, this is NaN. If a building is multi-use, and some of the building's uses have a compliance threshold, but others don't, use the expected GHGI (greenhouse gas emissions intensity) if nothing is changed as the GHGI for the portion of the building that is not subject to BEPS yet.
    - Example: A building is 50% retail and 50% multifamily housing. The target for retail in 2033 is 1.03 and there is no target for multifamily housing. This building would have a GHGI of 4.0 in 2033 if no changes were made to it. We estimate the GHGHI target as `(0.5 * 1.03) + (0.5 * 4.0) = 2.515`. 
    - NB: These numbers will be different than the RMI calculations. The RMI model mistakenly used a GHGI of zero for parts of buildings that are not yet subject to BEPS. So in the example above, RMI's model would list the GHGI target as`0.5 * 1.03 + 0.5 * 0 = 0.515`, which leaves out half the building. Then the target GHGI would actually go **up** when the multifamily part of the building became subject to BEPS! This model corrects that error.
- `expected_baseline`: The expected emissions if nothing is changed about the building, as calculated by the sum of: `total use energy for type * energy emissions factor for energy type` for the three energy types (col C)
- `expected_baseline_ghgi`: The expected GHGI if nothing is changed about the building (col B), as calculated by the `expected emissions / total GFA`
- `compliant_ghgi`: The expected GHGI if the building is compliant with BEPS, as defined by (col H):
    - if the BEPS GHGI target is lower than the expected GHGI, use the BEPS GHGI target
    - if the expected GHGI is lower than the BEPS GHGI target, use the expected GHGI
- `compliant_emissions`: The expected emissions if the building is compliant with BEPS, as defined by the `compliant GHGI * total GFA` (col J)
- `compliance_status`: Whether or not the building is compliant (col K):
    - yes: the baseline GHGI is lower than the expected compliant GHGI for this year
    - no: the baseline GHGI is higher than the expected compliant GHGI for this year
    - no requirement yet: the building doesn't have a compliance requirement for this year
- `compliance_fees`: Noncompliance fees. For years where buildings will be taxed for being noncompliant, this is `$2.50 * total GFA`    

In [1]:
import pandas as pd
import numpy as np

In [2]:
from baseline_model import BaselineBEPSModel

In [3]:
pd.options.mode.chained_assignment = None

In [4]:
JAN_TARGETS_PATH = '../data/input_data/jan_proposal_emissions_targets.csv'
JUNE_TARGETS_PATH = '../data/input_data/june_proposal_emissions_targets.csv'
EMISSIONS_PATH = '../data/input_data/energy_emissions.csv'
BUILDING_DATA_PATH = '../data/input_data/cleaned_building_data_with_policy_gfa.csv'

JAN_FINE_YEARS = [2027, 2030, 2035, 2040, 2045, 2050]
JUNE_FINE_YEARS = [2030, 2035, 2040, 2045, 2050]
FINE_PER_SQ_FT = 2.5

## January Proposal

In [5]:
jan_model = BaselineBEPSModel(EMISSIONS_PATH, JAN_TARGETS_PATH, BUILDING_DATA_PATH, JAN_FINE_YEARS, FINE_PER_SQ_FT)

In [6]:
jan_model.calculate_baseline_model(2027,2050)

Model calculations complete. Access the model dataframe as model_name.scenario_results


In [18]:
jan_model.scenario_results.head()

,OSEBuildingID,BuildingName,Total GFA for Policy,sq_ft_classification,LargestPropertyUseType OSE,SecondLargestPropertyUseType OSE,ThirdLargestPropertyUseType OSE,year,expected_baseline,expected_baseline_ghgi,city_ghgi_target,compliant_ghgi,compliant_emissions,compliance_status,compliance_fees
0,1,MAYFLOWER PARK HOTEL,88434.0,C,Hotel,NaN,NaN,2027.0,2.705014e+05,3.058794,3.058794,3.058794,2.705014e+05,No,221085.0
1,2,PARAMOUNT HOTEL,103566.0,B,Hotel,NaN,Restaurant,2027.0,2.843616e+05,2.745704,2.059096,2.059096,2.132523e+05,No,258915.0
2,3,WESTIN HOTEL (Parent Building),895128.0,A,Hotel,NaN,Recreation,2027.0,2.193901e+06,2.450936,1.876172,1.876172,1.679414e+06,No,2237820.0
3,5,HOTEL MAX,61320.0,C,Hotel,NaN,NaN,2027.0,2.934861e+05,4.786141,4.786141,4.786141,2.934861e+05,No,153300.0
4,8,WARWICK SEATTLE HOTEL,191454.0,B,Hotel,NaN,Recreation,2027.0,5.827520e+05,3.043822,1.431403,1.431403,2.740479e+05,No,478635.0


## Summary stats for January Proposal

In [7]:
jan_model.get_total_emissions_by_year()

Emissions by year calculations complete. Access the annual emissions dataframe as model_name.emissions_by_year


In [8]:
jan_model.emissions_by_year

,compliant_emissions
year,
2027.0,3.335484e+08
2028.0,3.225803e+08
2029.0,3.067583e+08
2030.0,2.780274e+08
2031.0,2.462242e+08
2032.0,2.277874e+08
2033.0,2.196453e+08
2034.0,2.100763e+08
2035.0,1.975086e+08


In [9]:
# Calculate percent of CO2 reduction by 2040

jan_model.get_percent_emissions_reduction_by_given_year(2040)

0.8637739067995087

## June proposal

In [13]:
june_model = BaselineBEPSModel(EMISSIONS_PATH, JUNE_TARGETS_PATH, BUILDING_DATA_PATH, JUNE_FINE_YEARS, FINE_PER_SQ_FT)

In [14]:
june_model.calculate_baseline_model(2027,2050)

Model calculations complete. Access the model dataframe as model_name.scenario_results


In [15]:
june_model.scenario_results.head()

,OSEBuildingID,BuildingName,Total GFA for Policy,sq_ft_classification,LargestPropertyUseType OSE,SecondLargestPropertyUseType OSE,ThirdLargestPropertyUseType OSE,year,expected_baseline,expected_baseline_ghgi,city_ghgi_target,compliant_ghgi,compliant_emissions,compliance_status,compliance_fees
0,1,MAYFLOWER PARK HOTEL,88434.0,C,Hotel,NaN,NaN,2027.0,2.705014e+05,3.058794,3.058794,3.058794,2.705014e+05,No,0.0
1,2,PARAMOUNT HOTEL,103566.0,B,Hotel,NaN,Restaurant,2027.0,2.843616e+05,2.745704,2.346333,2.346333,2.430003e+05,No,0.0
2,3,WESTIN HOTEL (Parent Building),895128.0,A,Hotel,NaN,Recreation,2027.0,2.193901e+06,2.450936,2.071342,2.071342,1.854116e+06,No,0.0
3,5,HOTEL MAX,61320.0,C,Hotel,NaN,NaN,2027.0,2.934861e+05,4.786141,4.786141,4.786141,2.934861e+05,No,0.0
4,8,WARWICK SEATTLE HOTEL,191454.0,B,Hotel,NaN,Recreation,2027.0,5.827520e+05,3.043822,1.962584,1.962584,3.757446e+05,No,0.0


## Summary stats for June

In [16]:
june_model.get_total_emissions_by_year()

Emissions by year calculations complete. Access the annual emissions dataframe as model_name.emissions_by_year


In [17]:
june_model.emissions_by_year

,compliant_emissions
year,
2027.0,8.566706e+08
2028.0,8.566706e+08
2029.0,8.566706e+08
2030.0,8.260823e+08
2031.0,3.333222e+08
2032.0,3.077036e+08
2033.0,2.980300e+08
2034.0,2.836252e+08
2035.0,2.626939e+08


In [19]:
june_model.get_percent_emissions_reduction_by_given_year(2040)

0.8157304067077129

## Comparing January and June

In [25]:
june_model.scenario_results['compliant_emissions'].sum()

6375484698.434601

In [26]:
jan_model.scenario_results['compliant_emissions'].sum()

3296041504.5541496

That's twice as much emissions in June vs January. Something is wrong. Let's look at the original RMI building spreadsheet.

## Proposals with Original RMI Spreadsheet

Something is wrong with the numbers we see above. Let's run the numbers again with the original RMI spreadsheet to verify that our model works correctly. If so, we can dig into what is wrong with the numbers we calculated for the public city data.

We'll rerun the numbers with the original RMI spreadsheet. Two things are added to this spreadsheet:

- building classification (A, B, C, etc. based on GFA that is subject to the policy)
- updated column names to match the model: 'Total_GFA' -> 'Total GFA for Policy', 'Steam(kBtu)' -> 'SteamUse(kBtu)', and 'percent_sqft_1st' -> 'LargestPropertyUseType Percent GFA', etc.

### January model

In [21]:
ORIG_BUILDING_DATA_PATH = '../data/input_data/rmi_building_analysis_with_new_col_names.csv'

In [15]:
orig_jan_model = BaselineBEPSModel(EMISSIONS_PATH, JAN_TARGETS_PATH, ORIG_BUILDING_DATA_PATH, JAN_FINE_YEARS, FINE_PER_SQ_FT)

In [16]:
orig_jan_model.calculate_baseline_model(2027, 2050)

Model calculations complete. Access the model dataframe as model_name.scenario_results


In [17]:
# Total emissions (kg) for Jan

orig_jan_model.scenario_results['compliant_emissions'].sum()

3528352321.3520036

In [18]:
# Percent reduction by 2040

orig_jan_model.get_percent_emissions_reduction_by_given_year(2040)

0.7042817698166167

In [23]:
orig_jan_model.get_total_emissions_by_year()

Emissions by year calculations complete. Access the annual emissions dataframe as model_name.emissions_by_year


In [24]:
orig_jan_model.emissions_by_year

,compliant_emissions
year,
2027.0,3.681950e+08
2028.0,3.448818e+08
2029.0,3.271189e+08
2030.0,2.909694e+08
2031.0,2.669814e+08
2032.0,2.478258e+08
2033.0,2.299096e+08
2034.0,2.142716e+08
2035.0,1.981371e+08


### June proposal

In [25]:
orig_june_model = BaselineBEPSModel(EMISSIONS_PATH, JUNE_TARGETS_PATH, ORIG_BUILDING_DATA_PATH, JUNE_FINE_YEARS, FINE_PER_SQ_FT)

In [26]:
orig_june_model.calculate_baseline_model(2027, 2050)

Model calculations complete. Access the model dataframe as model_name.scenario_results


In [27]:
orig_june_model.scenario_results['compliant_emissions'].sum()

4995307631.709463

In [28]:
orig_june_model.get_percent_emissions_reduction_by_given_year(2040)

0.6095068497074922

In [29]:
orig_june_model.get_total_emissions_by_year()

Emissions by year calculations complete. Access the annual emissions dataframe as model_name.emissions_by_year


In [30]:
orig_june_model.emissions_by_year

,compliant_emissions
year,
2027.0,4.561920e+08
2028.0,4.561920e+08
2029.0,4.561920e+08
2030.0,4.204626e+08
2031.0,3.699037e+08
2032.0,3.397445e+08
2033.0,3.181345e+08
2034.0,3.017397e+08
2035.0,2.703833e+08


### Difference between January and June

How much less CO2 (in kgs) will be released with the original January plan vs. the original June plan?

In [31]:
1 - (orig_jan_model.scenario_results['compliant_emissions'].sum() / orig_june_model.scenario_results['compliant_emissions'].sum())

0.29366666049663237